In [1]:
#Import Required Libraries And Required Inputs

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from datetime import datetime
import numpy as np
import string, os
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 5
learning_rate = 3e-4
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

In [3]:
device

'cpu'

In [4]:
# Read File& Required Preprocessing

In [5]:
with open('theatre.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
for punc in string.punctuation:
    text = text.replace(punc,' ')
txt = " ".join(t for t in text.replace('\n',' ').split(' ')).lower()  # Remove punctuation and convert to lower case
txt = txt.encode("utf8").decode("ascii",'ignore')

In [7]:
txt = [element for element in txt.split(' ') if not len(element) == 0]

In [8]:
# here are all the unique characters that occur in this text
words = sorted(list(set(txt)))
vocab_size = len(words)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(words) }
itos = { i:ch for i,ch in enumerate(words) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
#encode = lambda s: [stoi[c] for c in s.split(" ")] # encoder: take a string, output a list of integers
#decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [9]:
# Train and test splits

In [10]:
# perform tokenization
data = torch.tensor(encode(txt), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [14]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

19.548608 M parameters


In [14]:
for iter in range(max_iters):
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
torch.save(model.state_dict(), "Next Word Predictor_EPOCH400_v31.h5")


step 0: train loss 9.4020, val loss 9.4044
Current Time = 16:28:26
step 5: train loss 8.4342, val loss 8.4758
Current Time = 16:30:54
step 10: train loss 7.8603, val loss 7.9452
Current Time = 16:33:16
step 15: train loss 7.3761, val loss 7.4943
Current Time = 16:35:40
step 20: train loss 7.0285, val loss 7.1863
Current Time = 16:38:00
step 25: train loss 6.8148, val loss 7.0016
Current Time = 16:40:21
step 30: train loss 6.7159, val loss 6.9178
Current Time = 16:42:42
step 35: train loss 6.6811, val loss 6.9009
Current Time = 16:45:04
step 40: train loss 6.6768, val loss 6.9159
Current Time = 16:47:25
step 45: train loss 6.6744, val loss 6.9311
Current Time = 16:49:45
step 50: train loss 6.6659, val loss 6.9329
Current Time = 16:52:07
step 55: train loss 6.6533, val loss 6.9402
Current Time = 16:54:28
step 60: train loss 6.6380, val loss 6.9279
Current Time = 16:56:50
step 65: train loss 6.5943, val loss 6.9009
Current Time = 16:59:11
step 70: train loss 6.5355, val loss 6.8436
Curren

step 600: train loss 2.6657, val loss 7.5522
Current Time = 21:23:41
step 605: train loss 2.6338, val loss 7.5889
Current Time = 21:26:14
step 610: train loss 2.5872, val loss 7.6188
Current Time = 21:28:46
step 615: train loss 2.5465, val loss 7.6258
Current Time = 21:31:17
step 620: train loss 2.5082, val loss 7.6632
Current Time = 21:33:47
step 625: train loss 2.4702, val loss 7.6901
Current Time = 21:36:19
step 630: train loss 2.4323, val loss 7.7024
Current Time = 21:38:50
step 635: train loss 2.4067, val loss 7.7265
Current Time = 21:41:19
step 640: train loss 2.3677, val loss 7.7477
Current Time = 21:43:49
step 645: train loss 2.3242, val loss 7.7972
Current Time = 21:46:21
step 650: train loss 2.3028, val loss 7.7992
Current Time = 21:48:50
step 655: train loss 2.2557, val loss 7.8223
Current Time = 21:51:19
step 660: train loss 2.2164, val loss 7.8744
Current Time = 21:53:48
step 665: train loss 2.1879, val loss 7.8906
Current Time = 21:56:16
step 670: train loss 2.1466, val l

step 1195: train loss 0.4002, val loss 10.0925
Current Time = 02:21:43
step 1200: train loss 0.3968, val loss 10.0621
Current Time = 02:24:12
step 1205: train loss 0.3957, val loss 10.0822
Current Time = 02:26:41
step 1210: train loss 0.3878, val loss 10.1474
Current Time = 02:29:11
step 1215: train loss 0.3852, val loss 10.1154
Current Time = 02:31:39
step 1220: train loss 0.3807, val loss 10.0874
Current Time = 02:34:07
step 1225: train loss 0.3785, val loss 10.1874
Current Time = 02:36:36
step 1230: train loss 0.3777, val loss 10.2047
Current Time = 02:39:04
step 1235: train loss 0.3731, val loss 10.1072
Current Time = 02:41:31
step 1240: train loss 0.3666, val loss 10.1593
Current Time = 02:43:58
step 1245: train loss 0.3628, val loss 10.1703
Current Time = 02:46:25
step 1250: train loss 0.3588, val loss 10.2236
Current Time = 02:48:52
step 1255: train loss 0.3593, val loss 10.1485
Current Time = 02:51:22
step 1260: train loss 0.3547, val loss 10.2077
Current Time = 02:53:49
step 1

step 1775: train loss 0.2206, val loss 11.0479
Current Time = 07:03:11
step 1780: train loss 0.2200, val loss 11.1028
Current Time = 07:05:35
step 1785: train loss 0.2183, val loss 11.1005
Current Time = 07:07:58
step 1790: train loss 0.2179, val loss 11.1151
Current Time = 07:10:22
step 1795: train loss 0.2175, val loss 11.1098
Current Time = 07:12:46
step 1800: train loss 0.2173, val loss 11.1151
Current Time = 07:15:10
step 1805: train loss 0.2164, val loss 11.1296
Current Time = 07:17:34
step 1810: train loss 0.2174, val loss 11.0861
Current Time = 07:19:59
step 1815: train loss 0.2148, val loss 11.1425
Current Time = 07:22:23
step 1820: train loss 0.2141, val loss 11.1266
Current Time = 07:24:47
step 1825: train loss 0.2131, val loss 11.1343
Current Time = 07:27:10
step 1830: train loss 0.2117, val loss 11.2118
Current Time = 07:29:35
step 1835: train loss 0.2133, val loss 11.1146
Current Time = 07:31:59
step 1840: train loss 0.2116, val loss 11.1634
Current Time = 07:34:24
step 1

step 2355: train loss 0.1742, val loss 11.6413
Current Time = 11:39:11
step 2360: train loss 0.1725, val loss 11.6231
Current Time = 11:41:34
step 2365: train loss 0.1725, val loss 11.7026
Current Time = 11:43:56
step 2370: train loss 0.1709, val loss 11.6851
Current Time = 11:46:18
step 2375: train loss 0.1706, val loss 11.6862
Current Time = 11:48:41
step 2380: train loss 0.1710, val loss 11.6820
Current Time = 11:51:04
step 2385: train loss 0.1703, val loss 11.7448
Current Time = 11:53:27
step 2390: train loss 0.1720, val loss 11.7633
Current Time = 11:55:50
step 2395: train loss 0.1701, val loss 11.7576
Current Time = 11:58:13
step 2400: train loss 0.1699, val loss 11.7949
Current Time = 12:00:36
step 2405: train loss 0.1705, val loss 11.7330
Current Time = 12:02:59
step 2410: train loss 0.1690, val loss 11.7185
Current Time = 12:05:21
step 2415: train loss 0.1697, val loss 11.6615
Current Time = 12:07:42
step 2420: train loss 0.1702, val loss 11.7403
Current Time = 12:10:04
step 2

step 2935: train loss 0.1467, val loss 12.0660
Current Time = 16:10:55
step 2940: train loss 0.1471, val loss 12.1348
Current Time = 16:13:15
step 2945: train loss 0.1475, val loss 12.0506
Current Time = 16:15:35
step 2950: train loss 0.1474, val loss 12.1397
Current Time = 16:17:55
step 2955: train loss 0.1466, val loss 12.1211
Current Time = 16:20:14
step 2960: train loss 0.1465, val loss 12.0998
Current Time = 16:22:34
step 2965: train loss 0.1460, val loss 12.1961
Current Time = 16:24:55
step 2970: train loss 0.1469, val loss 12.1028
Current Time = 16:27:14
step 2975: train loss 0.1456, val loss 12.2032
Current Time = 16:29:34
step 2980: train loss 0.1485, val loss 12.1207
Current Time = 16:31:53
step 2985: train loss 0.1471, val loss 12.1704
Current Time = 16:34:14
step 2990: train loss 0.1468, val loss 12.1767
Current Time = 16:36:32
step 2995: train loss 0.1466, val loss 12.1381
Current Time = 16:38:52
step 3000: train loss 0.1453, val loss 12.1514
Current Time = 16:41:11
step 3

step 3515: train loss 0.1321, val loss 12.4707
Current Time = 20:39:52
step 3520: train loss 0.1306, val loss 12.5682
Current Time = 20:42:12
step 3525: train loss 0.1327, val loss 12.4993
Current Time = 20:44:31
step 3530: train loss 0.1307, val loss 12.5153
Current Time = 20:46:48
step 3535: train loss 0.1312, val loss 12.5445
Current Time = 20:49:08
step 3540: train loss 0.1312, val loss 12.5174
Current Time = 20:51:27
step 3545: train loss 0.1308, val loss 12.5526
Current Time = 20:53:45
step 3550: train loss 0.1318, val loss 12.4360
Current Time = 20:56:03
step 3555: train loss 0.1308, val loss 12.5265
Current Time = 20:58:23
step 3560: train loss 0.1303, val loss 12.4936
Current Time = 21:00:41
step 3565: train loss 0.1297, val loss 12.5707
Current Time = 21:03:01
step 3570: train loss 0.1296, val loss 12.5016
Current Time = 21:05:20
step 3575: train loss 0.1310, val loss 12.4830
Current Time = 21:07:39
step 3580: train loss 0.1295, val loss 12.4986
Current Time = 21:09:59
step 3

step 4095: train loss 0.1188, val loss 12.8323
Current Time = 01:14:05
step 4100: train loss 0.1188, val loss 12.8375
Current Time = 01:16:32
step 4105: train loss 0.1193, val loss 12.7884
Current Time = 01:18:58
step 4110: train loss 0.1179, val loss 12.8705
Current Time = 01:21:25
step 4115: train loss 0.1179, val loss 12.7625
Current Time = 01:23:50
step 4120: train loss 0.1192, val loss 12.7647
Current Time = 01:26:17
step 4125: train loss 0.1186, val loss 12.8456
Current Time = 01:28:42
step 4130: train loss 0.1204, val loss 12.7741
Current Time = 01:31:09
step 4135: train loss 0.1172, val loss 12.8141
Current Time = 01:33:34
step 4140: train loss 0.1191, val loss 12.7863
Current Time = 01:35:57
step 4145: train loss 0.1188, val loss 12.8245
Current Time = 01:38:21
step 4150: train loss 0.1185, val loss 12.8662
Current Time = 01:40:45
step 4155: train loss 0.1201, val loss 12.7764
Current Time = 01:43:11
step 4160: train loss 0.1191, val loss 12.8306
Current Time = 01:45:35
step 4

step 4675: train loss 0.1108, val loss 13.0774
Current Time = 05:55:11
step 4680: train loss 0.1104, val loss 13.1858
Current Time = 05:57:37
step 4685: train loss 0.1105, val loss 13.1142
Current Time = 06:00:03
step 4690: train loss 0.1096, val loss 13.1672
Current Time = 06:02:29
step 4695: train loss 0.1112, val loss 13.0749
Current Time = 06:04:53
step 4700: train loss 0.1115, val loss 13.1124
Current Time = 06:07:19
step 4705: train loss 0.1107, val loss 13.1220
Current Time = 06:09:45
step 4710: train loss 0.1106, val loss 13.1293
Current Time = 06:12:10
step 4715: train loss 0.1116, val loss 13.0927
Current Time = 06:14:36
step 4720: train loss 0.1092, val loss 13.1060
Current Time = 06:17:02
step 4725: train loss 0.1108, val loss 13.0930
Current Time = 06:19:26
step 4730: train loss 0.1099, val loss 13.1424
Current Time = 06:21:51
step 4735: train loss 0.1094, val loss 13.1162
Current Time = 06:24:15
step 4740: train loss 0.1102, val loss 13.1058
Current Time = 06:26:40
step 4

In [15]:
torch.load("Next Word Predictor_EPOCH400_v31.h5")

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.0017, -0.0828, -0.0049,  ...,  0.0091, -0.0147, -0.0039],
                      [ 0.0139,  0.0362, -0.0046,  ..., -0.0213,  0.0186, -0.0461],
                      [ 0.0651,  0.0144,  0.0237,  ..., -0.0483, -0.0168, -0.0025],
                      ...,
                      [ 0.0082, -0.0229,  0.0010,  ...,  0.0071, -0.0334, -0.0250],
                      [ 0.0495,  0.0679,  0.0529,  ...,  0.0258, -0.0249,  0.0179],
                      [ 0.0362,  0.0431,  0.0139,  ...,  0.0130, -0.0115, -0.0587]],
                     device='cuda:0')),
             ('position_embedding_table.weight',
              tensor([[-9.1582e-02, -2.1402e-01,  1.7561e-02,  ...,  4.4659e-02,
                       -7.2588e-02,  6.5580e-03],
                      [-1.7427e-03, -5.6298e-02, -9.3286e-03,  ...,  1.4173e-02,
                        9.9526e-03, -2.2528e-02],
                      [ 3.7378e-03,  1.1351e-02,  1.4113e-02,  ...,  1.

In [16]:
encode = lambda s: [stoi[c] for c in s.split(" ")] # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [46]:
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

In [29]:
text = "neighbour grass is always green".lower()
context = np.array(encode(text)).reshape(-1,len(text.split(' ')))[:3]
context =torch.tensor(context, dtype=torch.int32,device=device) 
answer = m.generate(context, max_new_tokens=10)[0].tolist()
decode(answer)

'neighbour grass is always green envy fiddler bond key sob easeful inherits unbarbed directed therewith'